# Google Cloud SQL for PostgreSQL

[Cloud SQL](https://cloud.google.com/sql) 是一项完全托管的关系型数据库服务，提供高性能、无缝集成以及出色的可扩展性，并支持 PostgreSQL 等数据库引擎。

本指南简要介绍了如何使用适用于 PostgreSQL 的 Cloud SQL 通过 `PostgresVectorStore` 类来存储向量嵌入。


## 概览

### 集成详情

| 类名               | 包名                                    | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/google_cloud_sql_pg/) | 包最新版本 |
| :------------------ | :----------------------------------------- | :--------------------------------------------------------------------------------------------: | :------------: |
| PostgresVectorStore | [`@langchain/google-cloud-sql-pg`](https://www.npmjs.com/package/@langchain/google-cloud-sql-pg) |                                               ✅                                               |     0.0.1      |


### 开始之前

为了使用此软件包，您首先需要完成以下步骤：

1.  [选择或创建一个云平台项目。](https://developers.google.com/workspace/guides/create-project)
2.  [为您的项目启用结算功能。](https://cloud.google.com/billing/docs/how-to/modify-project#enable_billing_for_a_project)
3.  [启用 Cloud SQL 管理 API。](https://console.cloud.google.com/flows/enableapi?apiid=sqladmin.googleapis.com)
4.  [设置身份验证。](https://cloud.google.com/docs/authentication)
5.  [创建一个 CloudSQL 实例](https://cloud.google.com/sql/docs/postgres/connect-instance-auth-proxy#create-instance)
6.  [创建一个 CloudSQL 数据库](https://cloud.google.com/sql/docs/postgres/create-manage-databases)
7.  [向数据库添加用户](https://cloud.google.com/sql/docs/postgres/create-manage-users)


### 身份验证

使用 ```gcloud auth login``` 命令对您的 Google Cloud 帐户进行本地身份验证。

### 设置您的 Google Cloud 项目

将您的 Google Cloud 项目 ID 设置为在本地使用 Google Cloud 资源：

In [ ]:
gcloud config set project YOUR-PROJECT-ID

如果不知道项目 ID，请尝试以下方法：
*   运行 `gcloud config list`。
*   运行 `gcloud projects list`。
*   查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

## 设置 PostgresVectorStore 实例

要使用 PostgresVectorStore 库，你需要安装 `@langchain/google-cloud-sql-pg` 包，然后按照以下步骤操作。

首先，你需要登录到你的 Google Cloud 账户，并根据你的 Google Cloud 项目设置以下环境变量；这些变量将根据你希望如何配置（fromInstance、fromEngine、fromEngineArgs）你的 PostgresEngine 实例而定义：


In [ ]:
PROJECT_ID="your-project-id"
REGION="your-project-region" // example: "us-central1"
INSTANCE_NAME="your-instance"
DB_NAME="your-database-name"
DB_USER="your-database-user"
PASSWORD="your-database-password"

### 设置实例

要实例化 PostgresVectorStore，首先需要通过 PostgresEngine 创建数据库连接，然后初始化向量存储表，最后调用 `.initialize()` 方法来实例化向量存储。


In [ ]:
import {
  Column,
  PostgresEngine,
  PostgresEngineArgs,
  PostgresVectorStore,
  PostgresVectorStoreArgs,
  VectorStoreTableArgs,
} from "@langchain/google-cloud-sql-pg";
import { SyntheticEmbeddings } from "@langchain/core/utils/testing"; // This is used as an Embedding service
import * as dotenv from "dotenv";

dotenv.config();

const peArgs: PostgresEngineArgs = {
  user: process.env.DB_USER ?? "",
  password: process.env.PASSWORD ?? "",
};

// PostgresEngine instantiation
const engine: PostgresEngine = await PostgresEngine.fromInstance(
  process.env.PROJECT_ID ?? "",
  process.env.REGION ?? "",
  process.env.INSTANCE_NAME ?? "",
  process.env.DB_NAME ?? "",
  peArgs
);

const vectorStoreArgs: VectorStoreTableArgs = {
  metadataColumns: [new Column("page", "TEXT"), new Column("source", "TEXT")],
};

// Vector store table initilization
await engine.initVectorstoreTable("my_vector_store_table", 768, vectorStoreArgs);
const embeddingService = new SyntheticEmbeddings({ vectorSize: 768 });

const pvectorArgs: PostgresVectorStoreArgs = {
  metadataColumns: ["page", "source"],
};

// PostgresVectorStore instantiation
const vectorStore = await PostgresVectorStore.initialize(
  engine,
  embeddingService,
  "my_vector_store_table",
  pvectorArgs
);


## 管理向量存储

### 向向量存储中添加文档

要将文档添加到向量存储中，可以选择传递或不传递ID来实现


In [ ]:
import { v4 as uuidv4 } from "uuid";
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { page: 0, source: "https://example.com" },
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { page: 1, source: "https://example.com" },
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { page: 2, source: "https://example.com" },
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { page: 3, source: "https://example.com" },
};

const documents = [document1, document2, document3, document4];

const ids = [uuidv4(), uuidv4(), uuidv4(), uuidv4()];

await vectorStore.addDocuments(documents, { ids: ids });


### 从向量存储中删除文档

您可以通过传入要删除的ID数组，从向量存储中删除一个或多个文档：


In [ ]:
// deleting a document
const id1 = ids[0];
await vectorStore.delete({ ids: [id1] });

// deleting more than one document
await vectorStore.delete({ ids: ids });


## 搜索文档

一旦创建了向量存储并添加了相关文档后，您很可能希望在链或代理运行期间对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按如下方式进行：


In [ ]:
const filter = `"source" = "https://example.com"`;

const results = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of results) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}


如果你想执行相似性搜索并获得相应的分数，可以运行：


In [ ]:
const filter = `"source" = "https://example.com"`;
const resultsWithScores = await vectorStore.similaritySearchWithScore(
  "biology",
  2,
  filter
);

for (const [doc, score] of resultsWithScores) {
  console.log(
    `* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`
  );
}


### 使用最大边际相关性搜索进行查询

最大边际相关性优化了与查询的相似性以及所选文档之间的多样性。


In [ ]:
const options = {
  k: 4,
  filter: `"source" = 'https://example.com'`,
};

const results = await vectorStoreInstance.maxMarginalRelevanceSearch("biology", options);

for (const doc of results) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}
